In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

In [77]:
dt_train=pd.read_csv('Poem_classification - train_data.csv')
dt_test=pd.read_csv('Poem_classification - test_data.csv')
dt=pd.concat([dt_train,dt_test])


In [78]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 991 entries, 0 to 149
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Genre   991 non-null    object
 1   Poem    987 non-null    object
dtypes: object(2)
memory usage: 23.2+ KB


In [79]:

dt.dropna(inplace=True)
dt.isnull().sum()

Genre    0
Poem     0
dtype: int64

In [80]:
dt.drop_duplicates(inplace=True)
dt.duplicated().sum()

np.int64(0)

In [121]:
from nltk.tokenize import word_tokenize
import string
import re
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import  WordNetLemmatizer


stop=set(stopwords.words('english'))
punt=set(string.punctuation)

lem=WordNetLemmatizer()

def preprocess(text):
    text=text.lower()
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'[^\w\s]', '', text)
    text=word_tokenize(text)
    text = BeautifulSoup(text, "html.parser").get_text() # removing html tags such as <br></br>
    text = " ".join([word for word in text.split() if word not in stop_words]) # removing stopwords
    text = " ".join(word for word in text.split() if len(word) > 2) # removing too short words
    cleaned=[]
    for word in text:
        if word not in stop and word not in  punt:

            lemmed=lem.lemmatize(word)
            cleaned.append(lemmed)
    return " ".join(cleaned)  
dt['clean_poem']=dt['Poem'].apply(preprocess)            
            
            

NameError: name 'BeautifulSoup' is not defined

In [122]:
dt

,Genre,Poem,clean_poem
1,Music,In the thick brushthey spend the...,thick brushthey spend hottest part day soaking...
2,Music,Storms are generous. ...,storm generous something easy surrender sittin...
3,Music,—After Ana Mendieta Did you carry around the ...,ana mendieta carry around matin star hold fore...
4,Music,for Aja Sherrard at 20The portent may itself ...,aja sherrard portent may memory wallace steven...
5,Music,"for Bob Marley, Bavaria, November 1980 Here i...",bob marley bavaria november brilliant morning ...
...,...,...,...
145,Environment,"To pick a tulip from the garden, the red one. ...",pick tulip garden red one put desk small blue ...
146,Environment,We are as clouds that veil the midnight moon; ...,cloud veil midnight moon restlessly speed glea...
147,Environment,"When pulled, the spider web took another form....",pulled spider web took another formthe bullsey...
148,Environment,Whose woods these are I think I know. His hous...,whose wood think know house village though


In [123]:
new_dt=dt[['Genre','clean_poem']]

In [124]:
new_dt

,Genre,clean_poem
1,Music,thick brushthey spend hottest part day soaking...
2,Music,storm generous something easy surrender sittin...
3,Music,ana mendieta carry around matin star hold fore...
4,Music,aja sherrard portent may memory wallace steven...
5,Music,bob marley bavaria november brilliant morning ...
...,...,...
145,Environment,pick tulip garden red one put desk small blue ...
146,Environment,cloud veil midnight moon restlessly speed glea...
147,Environment,pulled spider web took another formthe bullsey...
148,Environment,whose wood think know house village though


In [125]:
x=new_dt['clean_poem']
y=new_dt['Genre']

In [126]:
dt['Genre'].value_counts()

Genre
Environment    252
Music          250
Death          241
Affection      241
Name: count, dtype: int64

In [127]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
x_train_tf = tfidf.fit_transform(x_train)
y_test_tf  = tfidf.transform(x_test)


from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train_en=le.fit_transform(y_train)
y_test_en=le.transform(y_test)



In [135]:
# from sklearn.naive_bayes import MultinomialNB

# model = MultinomialNB()
# model.fit(x_train_cv, y_train_en)


MultinomialNB()

In [138]:
from sklearn.svm import LinearSVC
model = LinearSVC(class_weight='balanced', max_iter=1000)
model.fit(x_train_tf, y_train_en)


LinearSVC(class_weight='balanced')

In [139]:
pred=model.predict(y_test_tf)

In [140]:
from sklearn.metrics import accuracy_score
print("Accuracy : ", accuracy_score(y_test_en,pred))

Accuracy :  0.41116751269035534
